In [1]:
import pandas as pd
import re
from time import time
from pprint import pprint

In [2]:
from pyspark.mllib.fpm import FPGrowth

In [51]:
#config
datafile = '/home/test/Documents/DataCleansing/Transaction Classification Problem/dataset_5BCDE.csv'
processed_data_out='/home/test/Documents/DataCleansing/Transaction Classification Problem/Processed_1M.csv'
fp_tree_1M = '/home/test/Documents/DataCleansing/Transaction Classification Problem/fp_tree_1M.csv'

parts = 10
num_obs=10

In [7]:
num_rep='\d{4,}'

In [52]:
data = sc.textFile(datafile,minPartitions=parts)


header = data.first()
data = data.filter(lambda line: line != header)
# print(data.take(num_obs),'\n')
data = data.map(lambda x: x.split("|"))
# print(data.take(num_obs),'\n')
data = data.map(lambda x: x[1])
# print(data.take(num_obs),'\n')
data = data.map(lambda line:re.sub('[^A-Za-z0-9]+', ',', line))
print(data.take(num_obs),'\n')
data = data.map(lambda line:re.sub(num_rep, 'NUM', line))
print(data.take(num_obs),'\n')
data = data.map(lambda line: line.lower().strip().split(',')[:3])
print(data.take(num_obs),'\n')
data = data.map(lambda line: [elem+'_P'+str(idx+1) for (elem,idx) in zip(line,range(len(line)))])
print(data.take(num_obs),'\n')


['TD007411000169,TDS', 'Closure,proceeds,credited,to,TD039111000395,', 'VLT,TO,T09', 'CASH,RECEIPT', 'BNG,CS,LOADED,AS,AT,02,5,16', 'CASH,RECEIPT', 'CASH,RECEIPT', 'CASH,RECEIPT', 'CASH,RECEIPT', 'VLT,T1'] 

['TDNUM,TDS', 'Closure,proceeds,credited,to,TDNUM,', 'VLT,TO,T09', 'CASH,RECEIPT', 'BNG,CS,LOADED,AS,AT,02,5,16', 'CASH,RECEIPT', 'CASH,RECEIPT', 'CASH,RECEIPT', 'CASH,RECEIPT', 'VLT,T1'] 

[['tdnum', 'tds'], ['closure', 'proceeds', 'credited'], ['vlt', 'to', 't09'], ['cash', 'receipt'], ['bng', 'cs', 'loaded'], ['cash', 'receipt'], ['cash', 'receipt'], ['cash', 'receipt'], ['cash', 'receipt'], ['vlt', 't1']] 

[['tdnum_P1', 'tds_P2'], ['closure_P1', 'proceeds_P2', 'credited_P3'], ['vlt_P1', 'to_P2', 't09_P3'], ['cash_P1', 'receipt_P2'], ['bng_P1', 'cs_P2', 'loaded_P3'], ['cash_P1', 'receipt_P2'], ['cash_P1', 'receipt_P2'], ['cash_P1', 'receipt_P2'], ['cash_P1', 'receipt_P2'], ['vlt_P1', 't1_P2']] 



In [ ]:
datamodel = FPGrowth.train(data,minSupport=0.001,numPartitions=10)


In [9]:
data.saveAsTextFile(processed_data_out)

In [10]:
df=data.toDF()
# data.take(100)
data.getNumPartitions()

10

In [15]:
df.head(4),df.first()

([Row(_1='trf_P1', _2='ifo_P2', _3='crsg_P3'),
  Row(_1='fee_P1', _2='on_P2', _3='od_P3'),
  Row(_1='shortfall_P1', _2='iro_P2', _3='crsg_P3'),
  Row(_1='part_P1', _2='pymt_P2', _3='num_P3')],
 Row(_1='trf_P1', _2='ifo_P2', _3='crsg_P3'))

In [ ]:
result = datamodel.freqItemsets().collect()
print(len(result))
result[:10],result[-10:]

In [18]:
data_length = data.count()
data_length

999999

In [19]:
#creating list of fp_tree
fp_tree_list =[[item[0],item[1],round(100*item[1]/data_length,3)] for item in result]
len(fp_tree_list)
pprint(fp_tree_list[:5])

[[['num_P1'], 1644, 0.164],
 [['mobpymt_P1'], 9246, 0.925],
 [['mobpymt_P1', 'pp_P3'], 9238, 0.924],
 [['mobpymt_P1', 'pp_P3', '4qtlnum_P2'], 9238, 0.924],
 [['mobpymt_P1', '4qtlnum_P2'], 9246, 0.925]]


In [20]:
#create dictionary of Fp tree
fp_tree_dict = {}
i=0
for elem in result:
    i+=1
    fp_tree_dict[str(elem[0])]=[elem[1],round(100*elem[1]/data_length,2)]
# result[0][1]
# print(fp_tree_dict.items())

In [21]:
#creating dataframe of FP tree
import pandas as pd
fp_df = pd.DataFrame(fp_tree_list,columns=['tree_map', 'absolute_support', 'support_ratio'])
print(fp_df.head())

                          tree_map  absolute_support  support_ratio
0                         [num_P1]              1644          0.164
1                     [mobpymt_P1]              9246          0.925
2              [mobpymt_P1, pp_P3]              9238          0.924
3  [mobpymt_P1, pp_P3, 4qtlnum_P2]              9238          0.924
4         [mobpymt_P1, 4qtlnum_P2]              9246          0.925


In [22]:
fp_df['num_Ps']=fp_df['tree_map'].apply(lambda x:len(x))

In [23]:
fp_df.head(20)

,tree_map,absolute_support,support_ratio,num_Ps
0,[num_P1],1644,0.164,1
1,[mobpymt_P1],9246,0.925,1
2,"[mobpymt_P1, pp_P3]",9238,0.924,2
3,"[mobpymt_P1, pp_P3, 4qtlnum_P2]",9238,0.924,3
4,"[mobpymt_P1, 4qtlnum_P2]",9246,0.925,2
5,[cw_P1],2175,0.218,1
6,"[cw_P1, by_P2]",1088,0.109,2
7,[self_P2],16961,1.696,1
8,"[self_P2, cdb_P1]",6157,0.616,2
9,"[self_P2, cdb_P1, num_P3]",5684,0.568,3


In [24]:
fp_df.num_Ps.value_counts()

1    66
2    63
3    15
Name: num_Ps, dtype: int64

In [25]:
fp_df.groupby('num_Ps').agg('mean')

,absolute_support,support_ratio
num_Ps,,
1,37040.590909,3.704152
2,32883.587302,3.288444
3,40760.066667,4.076133


In [26]:
def create_elem(l,i):
    if len(l) > i:
        return l[i]
    else:
        return ''

In [27]:
fp_df['Key_1'] = fp_df['tree_map'].apply(lambda l:l[0])
fp_df['Key_2'] = fp_df['tree_map'].apply(create_elem,args=(1,))
fp_df['Key_3'] = fp_df['tree_map'].apply(create_elem,args=(2,))

print(len(fp_df[fp_df['Key_3'] != '']))
fp_df[fp_df['Key_3'] != '']

15


,tree_map,absolute_support,support_ratio,num_Ps,Key_1,Key_2,Key_3
3,"[mobpymt_P1, pp_P3, 4qtlnum_P2]",9238,0.924,3,mobpymt_P1,pp_P3,4qtlnum_P2
9,"[self_P2, cdb_P1, num_P3]",5684,0.568,3,self_P2,cdb_P1,num_P3
17,"[no_P2, chq_P1, num_P3]",1210,0.121,3,no_P2,chq_P1,num_P3
23,"[cwd_P1, self_P3, by_P2]",1395,0.140,3,cwd_P1,self_P3,by_P2
31,"[ifo_P3, num_P2, chq_P1]",3921,0.392,3,ifo_P3,num_P2,chq_P1
40,"[wd_P2, atm_P1, num_P3]",521516,52.152,3,wd_P2,atm_P1,num_P3
59,"[ngn_P3, revaluation_P1, of_P2]",3778,0.378,3,ngn_P3,revaluation_P1,of_P2
64,"[pos_P1, pur_P2, num_P3]",1276,0.128,3,pos_P1,pur_P2,num_P3
71,"[3qtl002_P3, web_P1, trf_P2]",3055,0.306,3,3qtl002_P3,web_P1,trf_P2
76,"[zar_P3, revaluation_P1, of_P2]",1292,0.129,3,zar_P3,revaluation_P1,of_P2


In [28]:

fp_df.to_csv(fp_tree_1M,sep = '|')

In [29]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
print(sc.parallelize([1, 2, 3, 4]).aggregate((0, 0), seqOp, combOp))
print(sc.parallelize([]).aggregate((0, 0), seqOp, combOp))
# |      (0, 0)

(10, 4)
(0, 0)


In [30]:
rdd = sc.parallelize([-5, -4, -3, -2, -1, 1, 2, 3, 4], 10)
add = lambda x, y: x + y
print(rdd.treeReduce(add))
print(rdd.treeReduce(add),1)
print(rdd.treeReduce(add),2)
print(rdd.treeReduce(add),5)
print(rdd.treeReduce(add),10)

-5
-5 1
-5 2
-5 5
-5 10


In [11]:
string_json = sc.parallelize(("""\
{"id":"123",\
"name":"judg"}""","""\
{"id":"124",\
"name":"judfrfgvfg"}"""))

In [12]:
s_json = spark.read.json(string_json)


In [13]:
s_json.collect()

[Row(id='123', name='judg'), Row(id='124', name='judfrfgvfg')]

In [15]:
s_json.show()

+---+----------+
| id|      name|
+---+----------+
|123|      judg|
|124|judfrfgvfg|
+---+----------+



In [17]:
s_json.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



In [18]:
# import types
from pyspark.sql.types import *

In [19]:
#generate csv
csv_rdd = sc.parallelize([
    (123,'katie', 19, 'brown'),
    (234,'jhgdfy',98,'red'),
    (232, 'simoe',23,'blue')
])

In [ ]:
#specify schema
sche